In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from typing import List
import json

In [ ]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("API_KEY not set")
else:
    print("API_KEY set")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
class Website:
    url: str
    title: str
    body: str
    link: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.link = [link for link in links if link]

    def get_content(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://www.github.com")
ed.link

In [ ]:
link_system_prompt = "You are provided with the link of links found on a webpage. \
                    Your are able to decide which of the links would be most relevant to include in a brochure about the company, \
                    such as links to an About page, or a Company page, or Career/Jobs pages. \n"
link_system_prompt += "You should respond in json as in this example:"
link_system_prompt += """
{
    "links":[
        {"type": "about page", "url": "https://www.full.url/goes/here/about"},
        {"type": "careers page", "url": "https://www.full.url/careers/here/about"},
    ]
}
"""

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"here is the list of links found on {website.url} - "
    user_prompt += "please decide which of these are relevent web links for a brochure about the company, respond wuth full https URL. Do not include Terms of Service , Privacy , email links. \n"
    user_prompt += "Links (some might be relative links): \n"
    user_prompt += "\n".join(website.link)
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
        response_format={"type": "json_object"},
    )
    result = response.choices[0].message.content  #OpenAI responds, the result is stored in a dictionary-like format.
    return json.loads(result)


get_links("https://www.github.com")

In [ ]:
def get_all_details(url):
    result = "Landing pages:\n"
    result += Website(url).get_content()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_content()
    return result
#make brochure
print(get_all_details("https://www.github.com"))